# import def func

In [26]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [93]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20):
    col_src = [
        "name", "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

from JSONData import tdx_hdf5_api as h5a
def get_tdx_all_from_h5(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d':
        dl='60'
    else:
        dl='200'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


def get_tdx_all_from_h5(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
    if end is not None:
        h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
            '_' + end.replace('-', '') + '_' + 'all'
    else:
        h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))


## init data

### get_tdx_all

In [28]:
h5_day=get_tdx_all_from_h5(resample='d')

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


### sina

In [29]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

lastb:8.5 

[2025-06-19 10:46:01,074] ERROR:tdx_hdf5_api.py(run:148): count10 remove lock


('2025-06-18',
 code
 600297    广汇汽车
 Name: name, dtype: object)

In [30]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/19/25 10:46:14'

code
688138   2025-06-19 10:44:58
600249   2025-06-19 10:44:57
Name: ticktime, dtype: datetime64[ns]

1.2864980816841125

#### 实时成交

In [31]:
def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [32]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:/Volumes/RamDisk/sina_MultiIndex_data.h5
['/all_30']
/Volumes/RamDisk/sina_MultiIndex_data.h5


(1069715, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-06-19 10:45:13,67.84,68.6,67.28,67.4,13064200,68.49
2025-06-19 10:44:58,67.83,68.6,67.28,67.4,13061900,68.06
2025-06-19 10:45:52,67.84,68.6,67.28,67.4,13085304,67.31


0.636

总量:4862.21 亿


ticktime    2025-06-20 00:00:00
close                 68.081505
volume                     0.29
vol                       13.09
amount                     8.91
Name: 603019, dtype: object

'2025-06-19'

#### 读取实时成交个股

In [33]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

((5405, 5),
         volume  name
 code                
 300308    1.48  中际旭创
 300468    1.06  四方精创
 300255    0.98  常山药业
 002463    0.97  沪电股份
 002104    0.91  恒宝股份
 300502    0.87   新易盛
 300100    0.85  双林股份
 300773    0.82   拉卡拉
 002015    0.82  协鑫能科
 002050    0.79  三花智控
 002384    0.77  东山精密
 300180    0.75  华峰超纤
 003040    0.67   楚天龙
 002366    0.65  融发核电
 300476    0.65  胜宏科技
 300674    0.63  宇信科技
 601606    0.60  长城军工
 300059    0.59  东方财富
 002371    0.58  北方华创
 002074    0.57  国轩高科)

##### 合并实时成交量及SinaDM

In [34]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### resample 3d

In [35]:
h5_3d = get_tdx_all_from_h5(resample='3d')
tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [57]:
tdx_3d.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
show_tdx_data(tdx_select)

(5182, 218)

Count:85 218


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000403,派林生物,2025-05-15,18.09,19.02,15.98,1.10,18.190156,16.16,0.00,1.3
000766,通化金马,2025-04-09,25.42,23.20,15.66,39.81,25.889541,15.66,0.00,20.2
002015,协鑫能科,2025-04-09,12.59,8.60,6.34,55.00,12.607970,6.34,1.20,1.2
002017,东信和平,2025-04-09,17.67,10.75,8.07,48.19,17.486919,8.07,28.00,12.3
002104,恒宝股份,2025-03-31,15.10,10.94,6.42,56.09,15.276582,6.42,12.10,26.0
002130,沃尔核材,2025-04-09,23.80,21.30,14.80,1.20,23.826751,14.80,1.20,0.1
002246,北化股份,2025-04-09,13.29,13.10,8.60,42.09,13.358939,8.60,1.20,17.2
002316,ST亚联,2025-04-09,4.99,4.44,3.09,28.91,5.023568,3.09,14.10,7.3
002365,永安药业,2025-03-31,25.60,22.39,8.18,1.30,25.781970,8.18,7.70,-1.1


In [59]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
tdx_3d.shape
t3d_code = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
show_tdx_data(t3d_code)

(5182, 218)

Count:85 218


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000403,派林生物,2025-05-15,18.09,19.02,15.98,1.10,18.190156,16.16,0.00,1.3
000766,通化金马,2025-04-09,25.42,23.20,15.66,39.81,25.889541,15.66,0.00,20.2
002015,协鑫能科,2025-04-09,12.59,8.60,6.34,55.00,12.607970,6.34,1.20,1.2
002017,东信和平,2025-04-09,17.67,10.75,8.07,48.19,17.486919,8.07,28.00,12.3
002104,恒宝股份,2025-03-31,15.10,10.94,6.42,56.09,15.276582,6.42,12.10,26.0
002130,沃尔核材,2025-04-09,23.80,21.30,14.80,1.20,23.826751,14.80,1.20,0.1
002246,北化股份,2025-04-09,13.29,13.10,8.60,42.09,13.358939,8.60,1.20,17.2
002316,ST亚联,2025-04-09,4.99,4.44,3.09,28.91,5.023568,3.09,14.10,7.3
002365,永安药业,2025-03-31,25.60,22.39,8.18,1.30,25.781970,8.18,7.70,-1.1


### resample W

In [60]:
h5_w = get_tdx_all_from_h5(resample='w')
tdx_w = cct.combine_dataFrame(h5_w,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [64]:
tdx_w.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select_w = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# show_tdx_data(tdx_select_w)

(5186, 219)

In [65]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
show_tdx_data(tw_code)

Count:163 219


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
600156,NaN,2025-01-10,NaN,6.39,4.05,1.20,NaN,4.31,21.70,1.2
000001,平安银行,2025-04-11,11.77,11.61,10.12,1.30,11.714974,10.12,1.30,1.2
000408,藏格矿业,2024-12-20,41.07,36.81,26.75,1.20,41.031813,26.75,0.00,0.0
000553,安道麦A,2025-04-18,7.00,7.27,5.22,1.20,7.056906,5.38,1.20,16.8
000631,顺发恒业,2025-04-11,2.95,3.42,2.43,1.20,2.936105,2.43,1.20,-1.1
000657,中钨高新,2025-01-03,11.99,11.53,8.50,1.20,11.976425,8.70,31.30,-0.9
000701,厦门信达,2025-01-03,6.14,5.70,3.71,1.20,6.156041,4.30,33.59,1.2
000766,通化金马,2024-11-29,25.42,20.34,13.59,34.00,25.889541,13.59,26.41,1.2
000803,山高环能,2025-01-03,5.99,6.13,4.32,34.91,6.095368,4.60,0.10,0.2


### resample M

In [42]:
h5_m = get_tdx_all_from_h5(resample='m')
tdx_m = cct.combine_dataFrame(h5_m,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [66]:
tm_code = tdx_m.query('lasth1d > lasth2d > lasth3d and lastp1d > ma51d')
show_tdx_data(tm_code)

Count:1155 218


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
600156,NaN,2024-09-30,NaN,5.48,2.71,54.31,NaN,3.24,0.0,37.50
000001,平安银行,2025-04-30,11.77,11.60,9.00,1.30,11.714974,10.12,-1.1,1.20
000010,美丽生态,2024-09-30,3.03,3.03,1.38,30.00,3.028547,1.64,-1.1,1.10
000014,沙河股份,2025-02-28,11.38,12.61,7.64,1.20,11.451882,9.77,-1.0,0.10
000017,深中华A,2025-02-28,6.73,7.12,5.06,1.20,6.731044,5.50,0.0,0.10
000020,深华发Ａ,2025-04-30,16.68,15.72,9.00,1.30,16.683077,9.22,-1.1,-0.90
000026,飞亚达,2025-01-27,11.73,10.56,8.12,1.20,11.658715,8.78,-1.1,31.50
000029,深深房Ａ,2025-01-27,19.75,18.05,10.00,1.30,19.864317,13.63,4.1,0.00
000159,国际实业,2025-04-30,6.03,7.44,4.83,1.30,5.976218,4.21,0.0,1.10


### resample D

In [67]:
h5_d = get_tdx_all_from_h5(resample='d')
tdx_d = cct.combine_dataFrame(h5_d,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [75]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d' )
show_tdx_data(td_code,limit=5)

Count:1710 220


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
600608,NaN,2025-04-28,NaN,3.97,3.10,0.0,NaN,2.88,-1.1,1.30
399006,NaN,2025-04-07,NaN,2084.00,1964.00,0.2,NaN,1757.00,-0.1,1.20
999999,NaN,2025-04-07,NaN,3404.00,3286.00,0.2,NaN,3040.00,-0.1,1.20
600156,NaN,2025-04-08,NaN,7.50,5.14,1.2,NaN,4.42,0.0,33.41
605123,派克新材,2025-04-07,67.35,67.81,53.88,27.3,68.360476,50.38,-0.1,0.10


In [86]:
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))

### check data

In [85]:
code_check = [co for co in code_f if co in tdx_3d.index]
len(code_check)

34

In [87]:
show_tdx_data(td_code.loc[code_f])

Count:34 220


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
002878,元隆雅图,2025-04-08,24.02,22.66,14.33,-1.00,23.604950,11.88,17.3,25.3
600287,ST舜天,2025-04-09,5.97,5.71,4.59,39.69,6.044551,3.94,1.3,1.2
603818,曲美家居,2025-04-08,4.15,4.65,2.92,38.91,4.246784,2.42,1.2,-1.1
837748,路桥信息,2025-04-07,71.25,84.50,24.78,-1.00,71.553245,19.09,0.1,0.0
002731,萃华珠宝,2025-04-08,15.05,14.88,10.08,-1.00,15.084798,8.24,17.3,12.3
002365,永安药业,2025-03-31,25.60,24.62,10.61,1.20,25.781970,8.18,17.3,-1.1
603585,苏利股份,2025-04-16,19.46,20.03,15.03,44.91,19.637387,13.43,1.2,-1.0
600259,广晟有色,2025-04-18,53.10,54.78,36.97,-0.90,53.435477,33.12,15.3,1.2
688719,爱科赛博,2025-04-07,42.65,39.03,25.88,1.20,43.301304,24.17,15.3,1.2


### Write toblk

In [94]:
write_code_to_blk(code_f,'077',rewrite=False)

write blk [Y] or [N]:y
Rewrite code [Y] or append [N](defalut:N):y
all write to /Users/Johnson/Documents/Johnson/WinTools/new_tdx2/T0002/blocknew/077.blk:34
all write to /Users/Johnson/Documents/Johnson/WinTools/new_tdx/T0002/blocknew/077.blk:34
write:34 block_path:/Users/Johnson/Documents/Johnson/WinTools/new_tdx2/T0002/blocknew/077.blk
